In [1]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.transforms.shared.raw.raw_contract import RawContract, download_s3_object
from core.dataset_contract import DatasetContract

db_transform = SessionHelper().session.query(Transformation).filter(Transformation.id == 13).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            state=state,
                            dataset="ingest_test")


2019-04-09 15:55:09,613 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-04-09 15:55:09,636 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-04-09 15:55:09,663 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-04-09 15:55:09,665 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-04-09 15:55:09,670 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-04-09 15:55:09,671 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-04-09 15:55:09,675 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating segment mocks.
2019-04-09 15:55:09,676 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pipeline type mocks.
2019-04-09 15:55:09,680 

# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [2]:
print (db_transform.variables)

namespace(delimiter='|', encoding='utf8', input_file_prefix='rommsa.SUN.INTEGRICHAIN_SUN_MCKESSON', skip_rows=2)


In [3]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
        delimiter: str = db_transform.variables.delimiter # The input file delimiter
        skip_rows: int = db_transform.variables.skip_rows # the number of rows to skip before reading
        encoding: str = db_transform.variables.encoding # The file encoding
        input_file_prefix: str = db_transform.variables.input_file_prefix # The prefix of input files to read in

## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead
transform = Transform()
transform.delimiter = ", "
transform.skip_rows = 0
transform.encoding = "utf8"
transform.input_file_prefix = "BRIOVA_"
transform.dataset_name = "init_ingest_patient_status"

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This transformation takes a collection of raw files based on location and their prefix, then creates a dataset from the data and outputs it to S3. The delimiter and encodings can be set, as well as a number of rows at the top to be skipped.

### Transformation

In [4]:
### Setup important outside imports here! (pandas, etc)
import pandas as pd
import os
import tempfile
from os import path
from encodings.aliases import aliases
from datetime import datetime
from core.logging import LoggerMixin, get_logger

In [5]:
### Setup for transformation object
ingest = Transform()
ingest.dataset = "ingest_test"
logger = get_logger("debug")

ingest.encoding = "utf_8"

input_contract = RawContract(branch=BRANCH_NAME,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            state="raw")
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

def set_initial_metadata(df, s3_url, run_timestamp):
    df['__metadata_original_s3_url'] = s3_url
    df['__metadata_ingest_timestamp'] = run_timestamp
    return df

def ingest_file(f, filename, input_contract):
    input_s3_url = input_contract.s3_path_with_filename(filename)
    logger.debug(f"Ingesting file: {filename}")
    print (filename)
    df = pd.read_csv(f, dtype="str", encoding=ingest.encoding, sep=ingest.delimiter, skiprows=ingest.skip_rows)
    df = set_initial_metadata(df, input_s3_url, run_timestamp)
    return df

valid_encodings = set(aliases.keys()) | set(aliases.values())
if ingest.encoding not in valid_encodings:
    invalid_encoding_message = f"initial_ingest_configuration {ingest.id} has invalid encoding: {ingest.encoding}"
    logger.critical(invalid_encoding_message)
    raise ValueError(invalid_encoding_message)
if len(ingest.delimiter) < 1:
    invalid_delimiter_message = f"initial_ingest_configuration {ingest.id} has no delimiter"
    logger.critical(invalid_delimiter_message)
    raise ValueError(invalid_delimiter_message)

# Input contract
in_files = input_contract.list_files(ingest.input_file_prefix)
run_timestamp = datetime.utcnow().strftime('"%Y-%m-%d %H:%M:%S"')

print (run_timestamp)

for f in in_files:
    filename = path.basename(f)
    with input_contract.download_raw_file(filename) as fn:
        df = ingest_file(fn, filename, input_contract)
        ingest.publish_contract.write_with_metadata(ingest.dataset, df, run_timestamp)
        print (ingest.publish_contract.s3_path)


### Publish

"2019-04-09 15:55:10"
rommsa.SUN.INTEGRICHAIN_SUN_MCKESSON_HUBCOPAYBRIDGE_20190401_01_556.txt.gz


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 15: invalid start byte

## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe,[])